# Simple Version 10

This notebook is an attempt to create a very simple version of the finite algebra code.

1. Adds **is_associative** & **is_commutative** tests to CaylyTable and Algebra classes, defined in Version 00

In [1]:
import numpy as np
import itertools as it
import pprint as pp

## Cayley Table

*"Named after the 19th-century British mathematician Arthur Cayley, a Cayley table describes the structure of a finite group by arranging all the possible products of all the group's elements in a square table reminiscent of an addition or multiplication table. Many properties of a group – such as whether or not it is abelian, which elements are inverses of which elements, and the size and contents of the group's center – can be discovered from its Cayley table."* [Wikipedia](https://en.wikipedia.org/wiki/Cayley_table)

Here, the **CayleyTable** class is used to represent a square array of indices, where the indices reference the positions of each of an algebra's elements within the list of all elements. For example, if there are 3 elements, [e, a, aa], then the corresponding instance of a CayleyTable will contain a 3x3 array where each array element is either a 0, 1, or 2.

In [41]:
class CayleyTable:
    """Represents a finite algebra's binary operation as a square array of integers, 0...n-1,
    where n is the order of the algebra, and the integers are indices, NOT algebraic elements.
    The indices denote the positions of the algebra's elements in a list."""
    
    def __init__(self, array):
        tmp = np.array(array, dtype=int)
        nrows, ncols = tmp.shape
        if nrows == ncols:
            if (np.min(tmp) >= 0) and (np.max(tmp) < nrows):
                self.__table = tmp
            else:
                raise ValueError(f"Array elements must be integers between 0 and {nrows - 1}, inclusive.")
        else:
            raise ValueError(f"The array must be square.")
    
    def __repr__(self):
        "Returns a cut-and-paste'able representation of the Cayley table."
        return f"{self.__class__.__name__}({self.__table.tolist()})"
    
    def __getitem__(self, tup):
        """Accesses a table element given its row & column indices"""
        row, col = tup
        return self.__table[row][col]
        
    @property
    def size(self):
        """Returns the number of rows/columnns of the table"""
        return self.__table.shape[0]
        
    def tolist(self):
        """Returns the table's nparray as a list of lists of ints."""
        return self.__table.tolist()
        
    @property
    def is_associative(self):
        """Returns True or False, depending on whether the table supports an associative
        binary operation."""
        indices = range(self.size)
        result = True
        for a in indices:
            for b in indices:
                for c in indices:
                    ab = self.__table[a][b]
                    bc = self.__table[b][c]
                    if not (self.__table[ab][c] == self.__table[a][bc]):
                        result = False
                        break
        return result
    
    @property
    def is_commutative(self):
        """Returns True or False, depending on whether the table supports a commutative
        binary operation."""
        n = self.size
        result = True
        for a in range(n):
            # Loop over the table's upper off-diagonal elements
            for b in range(a + 1, n):
                if self.__table[a][b] != self.__table[b][a]:
                    result = False
                    break
        return result

In [42]:
help(CayleyTable)

Help on class CayleyTable in module __main__:

class CayleyTable(builtins.object)
 |  CayleyTable(array)
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, tup)
 |      Accesses a table element given its row & column indices
 |  
 |  __init__(self, array)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Returns a cut-and-paste'able representation of the Cayley table.
 |  
 |  tolist(self)
 |      Returns the table's nparray as a list of lists of ints.
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |  
 |  is_associative
 |      Returns True or False, depending on whether the table supports an associative
 |      binary operation.
 |  
 |  is_commutative
 |      Returns True or False, depending on whether the table supports a commutative
 |      binary operation.
 |  
 |  size
 |      Returns the number of rows/columnns of the table
 |  
 |  --------------

In [3]:
arr0 = [[0, 1, 2], [1, 2, 0], [2, 0, 1]]
arr0

[[0, 1, 2], [1, 2, 0], [2, 0, 1]]

In [4]:
arr1 = [[0, 1, 2], [1, 2, 0]]
try:
    CayleyTable(arr1)
except Exception as msg:
    print(msg)

The array must be square.


In [5]:
arr2 = [[0, 1, 2], [1, 7, 0], [2, 0, 1]]
try:
    CayleyTable(arr2)
except Exception as msg:
    print(msg)

Array elements must be integers between 0 and 2, inclusive.


In [8]:
tbl0 = CayleyTable(arr0)
tbl0

CayleyTable([[0, 1, 2], [1, 2, 0], [2, 0, 1]])

In [9]:
tbl0.tolist()

[[0, 1, 2], [1, 2, 0], [2, 0, 1]]

In [10]:
tbl0.size

3

In [11]:
tbl0.is_associative

True

In [12]:
tbl0.is_commutative

True

## Binary Operator

The **BinaryOperator** class brings the list of elements (as strings) together with the corresponding CayleyTable.

In [13]:
class BinaryOperator:
    
    def __init__(self, elements, cayley_table):
        self.__elements = elements
        self.__table = cayley_table
    
    def __call__(self, elem1, elem2):
        row = self.__elements.index(elem1)
        col = self.__elements.index(elem2)
        index = self.__table[row, col]
        return self.__elements[index]
    
    @property
    def elements(self):
        return self.__elements
    
    @property
    def table(self):
        return self.__table

In [14]:
elems = ['e', 'a', 'aa']

In [15]:
op0 = BinaryOperator(elems, tbl0)

In [16]:
op0('e', 'a')

'a'

In [17]:
op0('a', 'a')

'aa'

In [18]:
op0('a', 'aa')

'e'

In [19]:
op0.elements

['e', 'a', 'aa']

In [20]:
op0.table

CayleyTable([[0, 1, 2], [1, 2, 0], [2, 0, 1]])

## Algebra

An **Algebra** consists of a set of elements with an associated binary operator, usually referred to as "multiplication" or "addition", depending on whether the operator is represented as $\times$ or $+$, resp.

In [22]:
class Algebra:
    
    def __init__(self, elements, array):
        self.__binop = BinaryOperator(elements, CayleyTable(array))
        
    def __getitem__(self, index):
        return self.__binop.elements[index]
    
    def __repr__(self):
        elems = self.__binop.elements
        tbl = self.__binop.table.tolist()
        return f"{self.__class__.__name__}(\n{elems},\n{tbl}\n)"
    
    @property
    def elements(self):
        return self.__binop.elements
    
    @property
    def order(self):
        return len(self.elements)
    
    @property
    def table(self):
        return self.__binop.table
    
    @property
    def is_associative(self):
        return self.__binop.table.is_associative
    
    @property
    def is_commutative(self):
        return self.__binop.table.is_commutative
    
    def op(self, elem1, elem2):
        """Use the algebra's binary operation to add/multiply two of it's elements,
        and return the sum/product.
        """
        return self.__binop(elem1, elem2)
    
    def __mul__(self, other):
        """Return the direct product of self with other. Other must be an algebra
        """
        dp_elements = list(it.product(self.elements, other.elements))  # cross-product of elements
        dp_table = list()  # start a new table
        for a in dp_elements:
            dp_table_row = list()  # Start a new row
            for b in dp_elements:
                dp_table_row.append(dp_elements.index((self.op(a[0], b[0]), other.op(a[1], b[1]))))
            dp_table.append(dp_table_row)  # Append the new row to the table
        return Algebra(list([f"{elem[0]}:{elem[1]}" for elem in dp_elements]),
                       dp_table)
    
    def info(self):
        print(f"\n** {self.__class__.__name__} **")
        print(f"Instance ID: {id(self)}")
        print(f"Order: {self.order}")
        print(f"Associative? {self.is_associative}")
        print(f"Commutative? {self.is_commutative}")
        print(f"Elements: {self.elements}")
        print("Table:")
        pp.pprint(self.table.tolist())
        return None

In [23]:
alg0 = Algebra(elems, arr0)

In [24]:
alg0

Algebra(
['e', 'a', 'aa'],
[[0, 1, 2], [1, 2, 0], [2, 0, 1]]
)

In [25]:
alg0.elements

['e', 'a', 'aa']

In [26]:
alg0.order

3

In [27]:
alg0.table

CayleyTable([[0, 1, 2], [1, 2, 0], [2, 0, 1]])

In [28]:
alg0.op('a', 'aa')

'e'

In [29]:
alg0.is_associative

True

In [30]:
alg0.is_commutative

True

In [31]:
alg0.info()


** Algebra **
Instance ID: 4402402576
Order: 3
Associative? True
Commutative? True
Elements: ['e', 'a', 'aa']
Table:
[[0, 1, 2], [1, 2, 0], [2, 0, 1]]


In [32]:
alg1 = alg0 * alg0
alg1

Algebra(
['e:e', 'e:a', 'e:aa', 'a:e', 'a:a', 'a:aa', 'aa:e', 'aa:a', 'aa:aa'],
[[0, 1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 0, 4, 5, 3, 7, 8, 6], [2, 0, 1, 5, 3, 4, 8, 6, 7], [3, 4, 5, 6, 7, 8, 0, 1, 2], [4, 5, 3, 7, 8, 6, 1, 2, 0], [5, 3, 4, 8, 6, 7, 2, 0, 1], [6, 7, 8, 0, 1, 2, 3, 4, 5], [7, 8, 6, 1, 2, 0, 4, 5, 3], [8, 6, 7, 2, 0, 1, 5, 3, 4]]
)

In [33]:
alg1.order

9

In [34]:
alg1.is_associative

True

In [35]:
alg1.is_commutative

True

In [36]:
alg1.info()


** Algebra **
Instance ID: 4677335888
Order: 9
Associative? True
Commutative? True
Elements: ['e:e', 'e:a', 'e:aa', 'a:e', 'a:a', 'a:aa', 'aa:e', 'aa:a', 'aa:aa']
Table:
[[0, 1, 2, 3, 4, 5, 6, 7, 8],
 [1, 2, 0, 4, 5, 3, 7, 8, 6],
 [2, 0, 1, 5, 3, 4, 8, 6, 7],
 [3, 4, 5, 6, 7, 8, 0, 1, 2],
 [4, 5, 3, 7, 8, 6, 1, 2, 0],
 [5, 3, 4, 8, 6, 7, 2, 0, 1],
 [6, 7, 8, 0, 1, 2, 3, 4, 5],
 [7, 8, 6, 1, 2, 0, 4, 5, 3],
 [8, 6, 7, 2, 0, 1, 5, 3, 4]]


## Elements

In [37]:
class Element:
    
    def __init__(self, elem_name, algebra):
        self.__algebra = algebra
        if isinstance(elem_name, str):
            if elem_name in self.__algebra:
                self.__name = elem_name
            else:
                raise ValueError(f"name must be an element of algebra")
        else:
            raise ValueError(f"name must be a string")

    def __repr__(self):
        return repr(self.__name)
    
    @property
    def name(self):
        return self.__name
    
    def __add__(self, other):
        elem = self.__algebra.op(self.__name, other.name)
        return Element(elem, self.__algebra)

In [38]:
def element_map(algebra):
    """Returns a dictionary where element names (str) are keys and the corresponding
    Element instances are the values."""
    return {elem: Element(elem, algebra) for elem in algebra.elements}

In [39]:
class Context:

    def __init__(self, algebra):
        self.element_map = element_map(algebra)

    def __enter__(self):
        return self.element_map

    def __exit__(self, _type, value, traceback):
        pass

In [40]:
with Context(alg1) as A:
    print(A['a:aa'] + A['aa:aa'] + A['a:a'])

'a:aa'
